In [1]:
%load_ext autoreload
%autoreload 2

import copy 
import sys
from pathlib import Path

sys.path.append('./')
sys.path.append('../')
sys.path.append('../..')

def show( object ):
  return [ x for x in dir( object ) if not x.startswith('__') ] 

    
import pprint 
import pandas as pd
#print( pd.__version__)  # must be 1.3.5 

import numpy as np
#print( np.__version__)  # must be 1.23.5

assert( str(pd.__version__) == '1.3.5')
assert( str(np.__version__) == '1.23.5')

from wf_lib2.crm_definitions import *
from wf_lib2.data.crm_dataset  import CRMDataset
from wf_lib2.data.crm_pattern  import CRMPattern
from wf_lib2.view_model.view_model import * 
import plotly.io as pio

print('Methods and properties of the CRMDataset class')
show( CRMDataset )


Methods and properties of the CRMDataset class


['_generate_dataset',
 '_get_distance_patterns_helper',
 '_slice_dates_dataset',
 'check_dataset',
 'copy_from',
 'filter_by',
 'from_pattern',
 'generate_default_dataset',
 'generate_default_multiwell_dataset',
 'get_column_names',
 'get_distance_patterns',
 'get_distances',
 'get_explicit_pattern',
 'get_pattern',
 'get_producer_injectors_distances_flat',
 'injector_locations',
 'injector_names',
 'instance',
 'mode_sampling_frequency',
 'producer_locations',
 'producer_names',
 'slice_coordinates_dataset',
 'slice_dates_dataset',
 'very_old_get_distance_patterns',
 'very_old_get_distance_patterns_helper']

In [2]:
dataset = CRMDataset.generate_default_multiwell_dataset() 
display(dataset.injectors_df.head(5))
display(dataset.producers_df.head(5))
display(dataset.locations_df)

w = dataset.producers_df.copy()
w['GAS_VOLUME'] = 0.1 * w['LIQUID_VOLUME']
w['OIL_VOLUME'] = 0.2 * w['LIQUID_VOLUME']
w['WATER_VOLUME'] = 0.1 * w['LIQUID_VOLUME']
dataset.producers_df = w 


1053 1053
Tying to guess the DATE format. dayforst ? False


,DATE,NAME,WATER_INJECTION_VOLUME,ZONE,SUBZONE,SECTOR,RESERVOIR
0,2022-01-23,Inj0,354.719228,Unique,Unique,0,Unique
1,2022-01-24,Inj0,354.716843,Unique,Unique,0,Unique
2,2022-01-25,Inj0,354.720018,Unique,Unique,0,Unique
3,2022-01-26,Inj0,354.720355,Unique,Unique,0,Unique
4,2022-01-27,Inj0,294.735906,Unique,Unique,0,Unique


,DATE,NAME,LIQUID_VOLUME,WATER_VOLUME,GAS_VOLUME,OIL_VOLUME,ZONE,SUBZONE,SECTOR,RESERVOIR
0,2022-01-23,Producer1,30.598382,30.598382,0.0,0.0,Unique,Unique,0,Unique
1,2022-01-24,Producer1,59.143985,59.143985,0.0,0.0,Unique,Unique,0,Unique
2,2022-01-25,Producer1,85.848004,85.848004,0.0,0.0,Unique,Unique,0,Unique
3,2022-01-26,Producer1,110.830344,110.830344,0.0,0.0,Unique,Unique,0,Unique
4,2022-01-27,Producer1,131.879565,131.879565,0.0,0.0,Unique,Unique,0,Unique


,NAME,X,Y,ZONE,SUBZONE,SECTOR,RESERVOIR,TYPE
0,Inj0,1000,0,Unique,Unique,0,Unique,Injector
1,Inj1,0,1000,Unique,Unique,0,Unique,Injector
2,Inj2,-1000,0,Unique,Unique,0,Unique,Injector
3,Inj3,0,-1000,Unique,Unique,0,Unique,Injector
4,Producer1,0,0,Unique,Unique,0,Unique,Producer
5,Producer2,500,500,Unique,Unique,0,Unique,Producer
6,Producer3,1000,1000,Unique,Unique,0,Unique,Producer


In [3]:
filters = {
    'zone': 'Unique', 'subzone':'Unique'
}
d= apply_dataset_filters( dataset, filters )
p = d.get_pattern(fix_time_gaps=True, fill_nan=0.0)

def fractions_plot( pattern, cummulative = False ):
    liquid_production_summary = pattern.liquid_production.sum(axis=1)
    water_production_summary  = pattern.water_production.sum(axis=1)
    oil_production_summary    = pattern.oil_production.sum(axis=1)
    gas_production_summary    = pattern.gas_production.sum(axis=1)
    liquid_production_summary
    
    fractions_dfs = [water_production_summary, oil_production_summary,gas_production_summary]
    fractions_labels = ['Water', 'Oil','Gas']
    fractions_data   = []
    sum_values = 1.0 
    last_date = water_production_summary.index.max() 

    for n, w in enumerate(fractions_dfs):
        if cummulative:
            w = w.cumsum(axis = 0 )  
            
        last_date_value = w.loc[ w.index.max() ]
        fractions_data.append(last_date_value)
        sum_values = sum_values + last_date_value 
        last_date = max( last_date, w.index.max() )
    for n in range( len(fractions_data )):
        fractions_data[n] = round( fractions_data[n] / sum_values, 1 ) 
    
        x = 'cummulative' if cummulative == True else '' 
        layout = copy.deepcopy( default_plotly_layout_python() )
        title = f'Volume fractions {x} present-day ({last_date.strftime("%d-%m-%Y")})'
        fig_dict  = {'data' : { 'values':fractions_data,'labels':fractions_labels, 'type':'pie','hole':0.4, 'name':'Fractions'}}
        fig_dict['layout'] = {'title':title}
        fig_dict['layout']['legend'] =dict(orientation='h',yanchor='bottom',y=-0.2,xanchor='center',x=0.5)
    return fig_dict

fig = fractions_plot( p, cummulative = False )
pio.show( fig )


We have a fix-time gaps instruction


In [14]:
dataset.injectors_df['DATE'].values.astype('datetime64[D]').astype( str )[0:10].tolist()  

w= dataset.injectors_df.copy()
w.set_index('DATE', inplace=True, drop=True)
w.index.values.astype('datetime64[D]').astype( str )[0:10].tolist()   

['2022-01-23',
 '2022-01-24',
 '2022-01-25',
 '2022-01-26',
 '2022-01-27',
 '2022-01-28',
 '2022-01-29',
 '2022-01-30',
 '2022-01-31',
 '2022-02-01']

In [ ]:



crm_dataset = CRMDataset.generate_default_multiwell_dataset()    
well_name = 'Producer1'

data = get_everything_for_this_well( crm_dataset, well_name, max_neighbours_distance = 2000, max_neighbours = 10 )

print( data.keys() )
data['rates']




1053 1053
Tying to guess the DATE format. dayforst ? False
351 351
Tying to guess the DATE format. dayforst ? False
dict_keys(['neighbours', 'rates', 'well_name', 'type'])


{'dates': ['2022-01-23',
  '2022-01-24',
  '2022-01-25',
  '2022-01-26',
  '2022-01-27',
  '2022-01-28',
  '2022-01-29',
  '2022-01-30',
  '2022-01-31',
  '2022-02-01',
  '2022-02-02',
  '2022-02-03',
  '2022-02-04',
  '2022-02-05',
  '2022-02-06',
  '2022-02-07',
  '2022-02-08',
  '2022-02-09',
  '2022-02-10',
  '2022-02-11',
  '2022-02-12',
  '2022-02-13',
  '2022-02-14',
  '2022-02-15',
  '2022-02-16',
  '2022-02-17',
  '2022-02-18',
  '2022-02-19',
  '2022-02-20',
  '2022-02-21',
  '2022-02-22',
  '2022-02-23',
  '2022-02-24',
  '2022-02-25',
  '2022-02-26',
  '2022-02-27',
  '2022-02-28',
  '2022-03-01',
  '2022-03-02',
  '2022-03-03',
  '2022-03-04',
  '2022-03-05',
  '2022-03-06',
  '2022-03-07',
  '2022-03-08',
  '2022-03-09',
  '2022-03-10',
  '2022-03-11',
  '2022-03-12',
  '2022-03-13',
  '2022-03-14',
  '2022-03-15',
  '2022-03-16',
  '2022-03-17',
  '2022-03-18',
  '2022-03-19',
  '2022-03-20',
  '2022-03-21',
  '2022-03-22',
  '2022-03-23',
  '2022-03-24',
  '2022-03-25',

In [ ]:

well_name = 'Inj1'
well_name = 'Producer1'
well_type, rates = None, None 
 
#these are the rates. 
if 1< 2: 
    if well_name in crm_dataset.injector_names:
        W_COL = find_column(w.injectors_df.columns, WATER_INJECTION_KEYS)
        N_COL = find_column(w.injectors_df.columns, NAME_KEYS)
        w = crm_dataset.filter_by(N_COL, [well_name] + crm_dataset.producer_names )
        p = w.injectors_df
        well_type = 'Injector'
        dates = p.index.values.astype('datetime64[D]').astype( str ).tolist()  if p.index.name == 'DATE' \
            else p['DATE'].values.astype('datetime64[D]').astype( str ).tolist() 
            
        water = p[W_COL].fillna(0.0).values.tolist() 
        rates = { 'dates':dates, 'Water_injection': water } 
        
    else:
        O_COL,G_COL,W_COL = find_column(w.producers_df.columns, OIL_PRODUCTION_KEYS), find_column(w.producers_df.columns, GAS_PRODUCTION_KEYS), find_column(w.producers_df.columns, WATER_PRODUCTION_KEYS)
        L_COL= find_column(w.producers_df.columns, LIQUID_PRODUCTION_KEYS)
        N_COL = find_column(w.injectors_df.columns, NAME_KEYS)
        well_type   = 'Producer'
        w = crm_dataset.filter_by(N_COL, [well_name] + crm_dataset.injector_names)
        p = w.producers_df
        
        dates = p.index.values.astype('datetime64[D]').astype( str ).tolist()  if p.index.name == 'DATE' \
            else p['DATE'].values.astype('datetime64[D]').astype( str ).tolist() 

        oil    = p[O_COL].fillna(0.0).values.tolist()
        water  = p[W_COL].fillna(0.0).values.tolist()
        gas    = p[G_COL].fillna(0.0).values.tolist()
        liquid = p[L_COL].fillna(0.0).values.tolist()
        
        rates = { 'dates':dates, 'Oil_production': oil, 'Gas_production': gas, 'Water_production': water, 'Liquid_production': liquid }
        
        
        well_types = {} 
        for iname in w.injector_names:
            well_types[iname] = 'Injector'
        for pname in w.producer_names:
            well_types[pname] = 'Producer'
            
 
flat_all_pairs_distances = crm_dataset.get_all_distances_flat(5654) 
result = get_closest_neighbors(flat_all_pairs_distances, well_name, N=10)
        
data = {'neighbours': result, 'rates':rates,
        'well_name': well_name,
        'type': well_type 
        
        }        

print('--------------------')
print( data['neighbours'] )
     

1053 1053
Tying to guess the DATE format. dayforst ? False
        NAME      X      Y    ZONE SUBZONE  SECTOR RESERVOIR      TYPE
0       Inj0   1000      0  Unique  Unique       0    Unique  Injector
1       Inj1      0   1000  Unique  Unique       0    Unique  Injector
2       Inj2  -1000      0  Unique  Unique       0    Unique  Injector
3       Inj3      0  -1000  Unique  Unique       0    Unique  Injector
4  Producer1      0      0  Unique  Unique       0    Unique  Producer
5  Producer2    500    500  Unique  Unique       0    Unique  Producer
6  Producer3   1000   1000  Unique  Unique       0    Unique  Producer
351 351
Tying to guess the DATE format. dayforst ? False
--------------------
[{'name': 'Producer2', 'type': 'Producer', 'distance': 707.1}, {'name': 'Inj2', 'type': 'Injector', 'distance': 1000.0}, {'name': 'Inj0', 'type': 'Injector', 'distance': 1000.0}, {'name': 'Inj3', 'type': 'Injector', 'distance': 1000.0}, {'name': 'Inj1', 'type': 'Injector', 'distance': 1000.0}, 